<div style='float: right'><img src='pic/paint.png'/></div>
## <div id='paint' />ペイントエリア

In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
from itertools import groupby, product
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
AABBC
DEFBC
DGFHH
DGIJH
KKLJH""".split()
nums = [[0,1,3], [3,2,2], [4,1,1]]
nw, nh = len(data[0]), len(data)

### 問題
* 盤面上にある、太線で区切られた部分（タイル）のいくつかを黒くぬります
* 盤面の数字は、その数字の入っているマスにタテヨコに隣り合うマスのうち、黒マスになるマスの数を表します
* 数字のマスが黒マスになることもあります
* どのタイルも、すべてのマスをぬるかすべてのマスをぬらずにおくかのどちらとし、タイルの一部のマスだけをぬってはいけません
* すべての黒マスはつながること
* 黒白マスとも、２×２以上はだめ

### 変数
* v：0:white, 1:black (1)

### 制約
* 2×2の黒がないこと (2)
* タイル内は同じこと (3)
* 数字は周りの黒の数と等しいこと (4)
* 全ての黒がつながること (5)

In [ ]:
m = LpProblem()
u = np.zeros((nh+2, nw+2), dtype=object)
v = u[1:-1,1:-1] = np.array(addbinvars(nh, nw)) # 0:white, 1:black (1)
for i, j in product(range(nh-1), range(nw-1)):
    m += lpSum(v[i:i+2,j:j+2]) <= 3 # (2)
for _, d in groupby(sorted(zip(''.join(data), v.flat)), lambda x:x[0]):
    d = list(d)
    for vi, vj in zip(d, d[1:]):
        m += vi[1] == vj[1] # (3)
w = u[1:-1,2:]+u[1:-1,:-2]+u[2:,1:-1]+u[:-2,1:-1]
for i, j, k in nums:
    m += lpSum(w[i,j]) == k # (4)
while True:
    %time m.solve()
    r = np.vectorize(value)(v)
    if unionfind.isconnected(r):
        break
    m += lpSum(v[r==0]) >= 1 # (5)
plt.imshow(1-r, cmap='gray', interpolation='none');